# Wheelchair Rugby Lineup Model\n\nThis notebook mirrors the logic in `model.py`. It aggregates four-player lineups,\nfilters by the 8-point rating cap, and ranks lineups by net goals per 60 minutes.

In [ ]:
from __future__ import annotations\n\nfrom dataclasses import dataclass\nfrom pathlib import Path\n\nimport pandas as pd\n\n\ndef resolve_data_dir() -> Path:\n    cwd = Path.cwd()\n    if (cwd / "data").exists():\n        return cwd / "data"\n    if (cwd.parent / "data").exists():\n        return cwd.parent / "data"\n    return cwd.parents[1] / "data"\n\n\nDATA_DIR = resolve_data_dir()\nPLAYER_CSV = DATA_DIR / "player_data.csv"\nSTINT_CSV = DATA_DIR / "stint_data.csv"

In [ ]:
@dataclass(frozen=True)\nclass LineupStat:\n    team: str\n    lineup: tuple[str, str, str, str]\n    rating_total: float\n    minutes: float\n    goals_for: float\n    goals_against: float\n\n    @property\n    def net_goals(self) -> float:\n        return self.goals_for - self.goals_against\n\n    @property\n    def net_per_60(self) -> float:\n        if self.minutes <= 0:\n            return 0.0\n        return self.net_goals * 60.0 / self.minutes\n\n\ndef _load_players(path: Path) -> dict[str, float]:\n    df = pd.read_csv(path)\n    return dict(zip(df["player"], df["rating"]))\n\n\ndef _lineup_rating(players: tuple[str, str, str, str], ratings: dict[str, float]) -> float:\n    return sum(ratings.get(p, 0.0) for p in players)\n\n\ndef _canonical_lineup(players: list[str]) -> tuple[str, str, str, str]:\n    return tuple(sorted(players))\n\n\ndef build_lineup_stats(\n    stint_df: pd.DataFrame,\n    ratings: dict[str, float],\n    max_rating: float = 8.0,\n) -> list[LineupStat]:\n    stats = {}\n\n    for row in stint_df.itertuples(index=False):\n        minutes = float(row.minutes)\n\n        home_players = [row.home1, row.home2, row.home3, row.home4]\n        away_players = [row.away1, row.away2, row.away3, row.away4]\n\n        for team, players, gf, ga in [\n            (row.h_team, home_players, row.h_goals, row.a_goals),\n            (row.a_team, away_players, row.a_goals, row.h_goals),\n        ]:\n            lineup = _canonical_lineup(players)\n            rating_total = _lineup_rating(lineup, ratings)\n            if rating_total > max_rating:\n                continue\n\n            key = (team, lineup)\n            if key not in stats:\n                stats[key] = {\n                    "minutes": 0.0,\n                    "goals_for": 0.0,\n                    "goals_against": 0.0,\n                    "rating_total": rating_total,\n                }\n            stats[key]["minutes"] += minutes\n            stats[key]["goals_for"] += float(gf)\n            stats[key]["goals_against"] += float(ga)\n\n    lineup_stats = []\n    for (team, lineup), agg in stats.items():\n        lineup_stats.append(\n            LineupStat(\n                team=team,\n                lineup=lineup,\n                rating_total=agg["rating_total"],\n                minutes=agg["minutes"],\n                goals_for=agg["goals_for"],\n                goals_against=agg["goals_against"],\n            )\n        )\n    return lineup_stats\n\n\ndef top_lineups(\n    lineup_stats: list[LineupStat],\n    min_minutes: float = 0.0,\n    top_n: int = 10,\n) -> dict[str, list[LineupStat]]:\n    by_team: dict[str, list[LineupStat]] = {}\n    for stat in lineup_stats:\n        if stat.minutes < min_minutes:\n            continue\n        by_team.setdefault(stat.team, []).append(stat)\n\n    for team, stats in by_team.items():\n        stats.sort(key=lambda s: (s.net_per_60, s.net_goals, s.minutes), reverse=True)\n        by_team[team] = stats[:top_n]\n    return by_team

In [ ]:
ratings = _load_players(PLAYER_CSV)\nstints = pd.read_csv(STINT_CSV)\nlineup_stats = build_lineup_stats(stints, ratings, max_rating=8.0)\n\nleaders = top_lineups(lineup_stats, min_minutes=1.0, top_n=5)\n\nrows = []\nfor team, stats in leaders.items():\n    for stat in stats:\n        rows.append(\n            {\n                "team": team,\n                "lineup": ", ".join(stat.lineup),\n                "rating": stat.rating_total,\n                "minutes": stat.minutes,\n                "net_goals": stat.net_goals,\n                "net_per_60": stat.net_per_60,\n            }\n        )\n\npd.DataFrame(rows).sort_values(["team", "net_per_60"], ascending=[True, False])